<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week15.do.txt --no_mako -->
<!-- dom:TITLE: Advanced machine learning and data analysis for the physical sciences -->

# Advanced machine learning and data analysis for the physical sciences
**Morten Hjorth-Jensen**, Department of Physics and Center for Computing in Science Education, University of Oslo, Norway

Date: **May 8, 2025**

## Plans for the week of May 5-9, 2025

**Deep generative models.**

1. Mathematics of diffusion models and selected examples

## Readings

**Reading on diffusion models.**

1. A central paper is the one by Sohl-Dickstein et al, Deep Unsupervised Learning using Nonequilibrium Thermodynamics, <https://arxiv.org/abs/1503.03585>

2. Calvin Luo at <https://arxiv.org/abs/2208.11970>

3. See also Diederik P. Kingma, Tim Salimans, Ben Poole, Jonathan Ho, Variational Diffusion Models, <https://arxiv.org/abs/2107.00630>

## Diffusion models, basics

Diffusion models are inspired by non-equilibrium thermodynamics. They
define a Markov chain of diffusion steps to slowly add random noise to
data and then learn to reverse the diffusion process to construct
desired data samples from the noise. Unlike VAE or flow models,
diffusion models are learned with a fixed procedure and the latent
variable has high dimensionality (same as the original data).

## Problems with probabilistic models

Historically, probabilistic models suffer from a tradeoff between two
conflicting objectives: \textit{tractability} and
\textit{flexibility}. Models that are \textit{tractable} can be
analytically evaluated and easily fit to data (e.g. a Gaussian or
Laplace). However, these models are unable to aptly describe structure
in rich datasets. On the other hand, models that are \textit{flexible}
can be molded to fit structure in arbitrary data. For example, we can
define models in terms of any (non-negative) function $\phi(\boldsymbol{x})$
yielding the flexible distribution $p\left(\boldsymbol{x}\right) =
\frac{\phi\left(\boldsymbol{x} \right)}{Z}$, where $Z$ is a normalization
constant. However, computing this normalization constant is generally
intractable. Evaluating, training, or drawing samples from such
flexible models typically requires a very expensive Monte Carlo
process.

## Diffusion models
Diffusion models have several interesting features
* extreme flexibility in model structure,

* exact sampling,

* easy multiplication with other distributions, e.g. in order to compute a posterior, and

* the model log likelihood, and the probability of individual states, to be cheaply evaluated.

## Original idea

In the original formulation, one uses a Markov chain to gradually
convert one distribution into another, an idea used in non-equilibrium
statistical physics and sequential Monte Carlo. Diffusion models build
a generative Markov chain which converts a simple known distribution
(e.g. a Gaussian) into a target (data) distribution using a diffusion
process. Rather than use this Markov chain to approximately evaluate a
model which has been otherwise defined, one can  explicitly define the
probabilistic model as the endpoint of the Markov chain. Since each
step in the diffusion chain has an analytically evaluable probability,
the full chain can also be analytically evaluated.

## Diffusion learning

Learning in this framework involves estimating small perturbations to
a diffusion process. Estimating small, analytically tractable,
perturbations is more tractable than explicitly describing the full
distribution with a single, non-analytically-normalizable, potential
function.  Furthermore, since a diffusion process exists for any
smooth target distribution, this method can capture data distributions
of arbitrary form.

## Mathematics of diffusion models

Let us go back our discussions of the variational autoencoders from the lecture of April 24, see
<https://github.com/CompPhysics/AdvancedMachineLearning/blob/main/doc/pub/week14/ipynb/week14.ipynb>. As
a first attempt at understanding diffusion models, we can think of
these as stacked VAEs, or better, recursive VAEs.

Let us try to see why. As an intermediate step, we consider so-called
hierarchical VAEs, which can be seen as a generalization of VAEs that
include multiple hierarchies of latent spaces.

**Note**: Many of the derivations and figures here are inspired and borrowed from the excellent exposition of diffusion models by Calvin Luo at <https://arxiv.org/abs/2208.11970>.

## Chains of VAEs

Markovian
VAEs represent a  generative process where we use  Markov chain to build a hierarchy of VAEs.

Each transition down the hierarchy is Markovian, where we decode each
latent set of variables $\boldsymbol{h}_t$ in terms of the previous latent variable $\boldsymbol{h}_{t-1}$.
Intuitively, and visually, this can be seen as simply stacking VAEs on
top of each other (see figure next slide).

One can think of such a model as a recursive VAE.

## Mathematical representation

Mathematically, we represent the joint distribution and the posterior
of a Markovian VAE as

$$
\begin{align*}
    p(\boldsymbol{x}, \boldsymbol{h}_{1:T}) &= p(\boldsymbol{h}_T)p_{\boldsymbol{\theta}}(\boldsymbol{x}|\boldsymbol{h}_1)\prod_{t=2}^{T}p_{\boldsymbol{\theta}}(\boldsymbol{h}_{t-1}|\boldsymbol{h}_{t})\\
    q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x}) &= q_{\boldsymbol{\phi}}(\boldsymbol{h}_1|\boldsymbol{x})\prod_{t=2}^{T}q_{\boldsymbol{\phi}}(\boldsymbol{h}_{t}|\boldsymbol{h}_{t-1})
\end{align*}
$$

## Back to the marginal probability

We can then define the marginal probability we want to optimize as

$$
\begin{align*}
\log p(\boldsymbol{x}) &= \log \int p(\boldsymbol{x}, \boldsymbol{h}_{1:T}) d\boldsymbol{h}_{1:T}  \\
&= \log \int \frac{p(\boldsymbol{x}, \boldsymbol{h}_{1:T})q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})} d\boldsymbol{h}_{1:T}         && \text{(Multiply by 1 = $\frac{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}$)}\\
&= \log \mathbb{E}_{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}\left[\frac{p(\boldsymbol{x}, \boldsymbol{h}_{1:T})}{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}\right]         && \text{(Definition of Expectation)}\\
&\geq \mathbb{E}_{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}\left[\log \frac{p(\boldsymbol{x}, \boldsymbol{h}_{1:T})}{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}\right]         && \text{(Discussed last week)}
\end{align*}
$$

## Diffusion models for hierarchical VAE, from <https://arxiv.org/abs/2208.11970>

A Markovian hierarchical Variational Autoencoder with $T$ hierarchical
latents.  The generative process is modeled as a Markov chain, where
each latent $\boldsymbol{h}_t$ is generated only from the previous latent
$\boldsymbol{h}_{t+1}$. Here $\boldsymbol{z}$ is our latent variable $\boldsymbol{h}$.

<!-- dom:FIGURE: [figures/figure1.png, width=800 frac=1.0] -->
<!-- begin figure -->

<img src="figures/figure1.png" width="800"><p style="font-size: 0.9em"><i>Figure 1: </i></p>
<!-- end figure -->

## Equation for the Markovian hierarchical VAE

We obtain then

$$
\begin{align*}
\mathbb{E}_{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}\left[\log \frac{p(\boldsymbol{x}, \boldsymbol{h}_{1:T})}{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}\right]
&= \mathbb{E}_{q_{\boldsymbol{\phi}}(\boldsymbol{h}_{1:T}|\boldsymbol{x})}\left[\log \frac{p(\boldsymbol{h}_T)p_{\boldsymbol{\theta}}(\boldsymbol{x}|\boldsymbol{h}_1)\prod_{t=2}^{T}p_{\boldsymbol{\theta}}(\boldsymbol{h}_{t-1}|\boldsymbol{h}_{t})}{q_{\boldsymbol{\phi}}(\boldsymbol{h}_1|\boldsymbol{x})\prod_{t=2}^{T}q_{\boldsymbol{\phi}}(\boldsymbol{h}_{t}|\boldsymbol{h}_{t-1})}\right]
\end{align*}
$$

We will modify this equation when we discuss what are normally called Variational Diffusion Models.

## Variational Diffusion Models

The easiest way to think of a Variational Diffusion Model (VDM) is as a Markovian Hierarchical Variational Autoencoder with three key restrictions:

1. The latent dimension is exactly equal to the data dimension

2. The structure of the latent encoder at each timestep is not learned; it is pre-defined as a linear Gaussian model.  In other words, it is a Gaussian distribution centered around the output of the previous timestep

3. The Gaussian parameters of the latent encoders vary over time in such a way that the distribution of the latent at final timestep $T$ is a standard Gaussian

The VDM posterior is

$$
\begin{align*}
    q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0) = \prod_{t = 1}^{T}q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})
\end{align*}
$$

## Second assumption

The distribution of each latent variable in the encoder is a Gaussian centered around its previous hierarchical latent.
Here then, the structure of the encoder at each timestep $t$ is not learned; it
is fixed as a linear Gaussian model, where the mean and standard
deviation can be set beforehand as hyperparameters, or learned as
parameters.

## Parameterizing Gaussian encoder

We parameterize the Gaussian encoder with mean $\boldsymbol{\mu}_t(\boldsymbol{x}_t) =
\sqrt{\alpha_t} \boldsymbol{x}_{t-1}$, and variance $\boldsymbol{\Sigma}_t(\boldsymbol{x}_t) =
(1 - \alpha_t) \textbf{I}$, where the form of the coefficients are
chosen such that the variance of the latent variables stay at a
similar scale; in other words, the encoding process is
variance-preserving.

Note that alternate Gaussian parameterizations
are allowed, and lead to similar derivations.  The main takeaway is
that $\alpha_t$ is a (potentially learnable) coefficient that can vary
with the hierarchical depth $t$, for flexibility.

## Encoder transitions

Mathematically, the encoder transitions are defined as

<!-- Equation labels as ordinary links -->
<div id="eq:27"></div>

$$
\begin{align*}
    q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1}) = \mathcal{N}(\boldsymbol{x}_{t} ; \sqrt{\alpha_t} \boldsymbol{x}_{t-1}, (1 - \alpha_t) \textbf{I}) \label{eq:27} \tag{1}
\end{align*}
$$

## Third assumption

From the third assumption, we know that $\alpha_t$ evolves over time
according to a fixed or learnable schedule structured such that the
distribution of the final latent $p(\boldsymbol{x}_T)$ is a standard Gaussian.
We can then update the joint distribution of a Markovian VAE to write
the joint distribution for a VDM as

$$
\begin{align*}
p(\boldsymbol{x}_{0:T}) &= p(\boldsymbol{x}_T)\prod_{t=1}^{T}p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t-1}|\boldsymbol{x}_t) \\
\text{where,}&\nonumber\\
p(\boldsymbol{x}_T) &= \mathcal{N}(\boldsymbol{x}_T; \boldsymbol{0}, \textbf{I})
\end{align*}
$$

## Noisification

Collectively, what this set of assumptions describes is a steady
noisification of an image input over time. We progressively corrupt an
image by adding Gaussian noise until eventually it becomes completely
identical to pure Gaussian noise.  See figure on next slide.

## Diffusion models, from <https://arxiv.org/abs/2208.11970>

<!-- dom:FIGURE: [figures/figure2.png, width=800 frac=1.0] -->
<!-- begin figure -->

<img src="figures/figure2.png" width="800"><p style="font-size: 0.9em"><i>Figure 1: </i></p>
<!-- end figure -->

## Gaussian modeling

Note that our encoder distributions $q(\boldsymbol{x}_t|\boldsymbol{x}_{t-1})$ are no
longer parameterized by $\boldsymbol{\phi}$, as they are completely modeled as
Gaussians with defined mean and variance parameters at each timestep.
Therefore, in a VDM, we are only interested in learning conditionals
$p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t-1}|\boldsymbol{x}_{t})$, so that we can simulate
new data.  After optimizing the VDM, the sampling procedure is as
simple as sampling Gaussian noise from $p(\boldsymbol{x}_T)$ and iteratively
running the denoising transitions
$p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t-1}|\boldsymbol{x}_{t})$ for $T$ steps to generate a
novel $\boldsymbol{x}_0$.

## Optimizing the variational diffusion model

$$
\begin{align*}
\log p(\boldsymbol{x})
&= \log \int p(\boldsymbol{x}_{0:T}) d\boldsymbol{x}_{1:T}\\
&= \log \int \frac{p(\boldsymbol{x}_{0:T})q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)} d\boldsymbol{x}_{1:T}\\
&= \log \mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\frac{p(\boldsymbol{x}_{0:T})}{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\right]\\
&\geq {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_{0:T})}{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\right]}\\
&= {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)\prod_{t=1}^{T}p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t-1}|\boldsymbol{x}_t)}{\prod_{t = 1}^{T}q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
&= {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)\prod_{t=2}^{T}p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t-1}|\boldsymbol{x}_t)}{q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})\prod_{t = 1}^{T-1}q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
&= {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)\prod_{t=1}^{T-1}p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t}|\boldsymbol{x}_{t+1})}{q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})\prod_{t = 1}^{T-1}q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
&= {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)}{q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})}\right] + \mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \prod_{t = 1}^{T-1}\frac{p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t}|\boldsymbol{x}_{t+1})}{q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
\end{align*}
$$

## Continues

$$
\begin{align*}
\log p(\boldsymbol{x})
&= {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)}{q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})}\right] + \mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \prod_{t = 1}^{T-1}\frac{p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t}|\boldsymbol{x}_{t+1})}{q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
&= {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)\right] + \mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)}{q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})}\right] + \mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[ \sum_{t=1}^{T-1} \log \frac{p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t}|\boldsymbol{x}_{t+1})}{q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
&= {\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)\right] + \mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)}{q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})}\right] + \sum_{t=1}^{T-1}\mathbb{E}_{q(\boldsymbol{x}_{1:T}|\boldsymbol{x}_0)}\left[ \log \frac{p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t}|\boldsymbol{x}_{t+1})}{q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
&= {\mathbb{E}_{q(\boldsymbol{x}_{1}|\boldsymbol{x}_0)}\left[\log p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)\right] + \mathbb{E}_{q(\boldsymbol{x}_{T-1}, \boldsymbol{x}_T|\boldsymbol{x}_0)}\left[\log \frac{p(\boldsymbol{x}_T)}{q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})}\right] + \sum_{t=1}^{T-1}\mathbb{E}_{q(\boldsymbol{x}_{t-1}, \boldsymbol{x}_t, \boldsymbol{x}_{t+1}|\boldsymbol{x}_0)}\left[\log \frac{p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t}|\boldsymbol{x}_{t+1})}{q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})}\right]}\\
\end{align*}
$$

## Interpretations

These equations can be interpreted as

* $\mathbb{E}_{q(\boldsymbol{x}_{1}|\boldsymbol{x}_0)}\left[\log p_{\boldsymbol{\theta}}(\boldsymbol{x}_0|\boldsymbol{x}_1)\right]$ can be interpreted as a **reconstruction term**, predicting the log probability of the original data sample given the first-step latent.  This term also appears in a vanilla VAE, and can be trained similarly.

* $\mathbb{E}_{q(\boldsymbol{x}_{T-1}|\boldsymbol{x}_0)}\left[D_{KL}(q(\boldsymbol{x}_T|\boldsymbol{x}_{T-1})\vert\vert p(\boldsymbol{x}_T))\right]$ is a **prior matching term**; it is minimized when the final latent distribution matches the Gaussian prior.  This term requires no optimization, as it has no trainable parameters; furthermore, as we have assumed a large enough $T$ such that the final distribution is Gaussian, this term effectively becomes zero.

## The last term

* $\mathbb{E}_{q(\boldsymbol{x}_{t-1}, \boldsymbol{x}_{t+1}|\boldsymbol{x}_0)}\left[D_{KL}(q(\boldsymbol{x}_{t}|\boldsymbol{x}_{t-1})\vert\vert p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t}|\boldsymbol{x}_{t+1}))\right]$ is a \textit{consistency term}; it endeavors to make the distribution at $\boldsymbol{x}_t$ consistent, from both forward and backward processes.  That is, a denoising step from a noisier image should match the corresponding noising step from a cleaner image, for every intermediate timestep; this is reflected mathematically by the KL Divergence.  This term is minimized when we train $p_{\theta}(\boldsymbol{x}_t|\boldsymbol{x}_{t+1})$ to match the Gaussian distribution $q(\boldsymbol{x}_t|\boldsymbol{x}_{t-1})$.

## Diffusion models, part 2, from <https://arxiv.org/abs/2208.11970>

<!-- dom:FIGURE: [figures/figure3.png, width=800 frac=1.0] -->
<!-- begin figure -->

<img src="figures/figure3.png" width="800"><p style="font-size: 0.9em"><i>Figure 1: </i></p>
<!-- end figure -->

## Optimization cost

The cost of optimizing a VDM is primarily dominated by the third term, since we must optimize over all timesteps $t$.

Under this derivation, all three terms are computed as expectations,
and can therefore be approximated using Monte Carlo estimates.
However, actually optimizing the ELBO using the terms we just derived
might be suboptimal; because the consistency term is computed as an
expectation over two random variables $\left\{\boldsymbol{x}_{t-1},
\boldsymbol{x}_{t+1}\right\}$ for every timestep, the variance of its Monte
Carlo estimate could potentially be higher than a term that is
estimated using only one random variable per timestep.  As it is
computed by summing up $T-1$ consistency terms, the final estimated
value may have high variance for large $T$ values.

## More details

For more details and implementaions, see Calvin Luo at <https://arxiv.org/abs/2208.11970>

<!-- FIGURE: [figures/figure4.png, width=800 frac=1.0] -->